# output.csv 파일에 pseudo labeling적용한 dataframe 생성

In [11]:
import numpy as np
import pandas as pd
import copy
import os

In [12]:
# set data_dir
data_dir = '/opt/ml/input/data/train_dataset'

In [15]:
# read saved output.csv 
csv_file_path = os.path.join(data_dir, 'output.csv')
test_predict = pd.read_csv(csv_file_path)
test_predict = test_predict['prediction']  

print('[  test_predict  ] : ', len(test_predict))

[  test_predict  ] :  744


In [3]:
#apply pseudo labeling
pseudo_labels=[]
for x in test_predict:
    pseudo_labels.append(1 if x>=0.5 else 0)
    
print('[  pseudo_labels  ] : ', len(pseudo_labels))

[  pseudo_labels  ] :  744


In [18]:
test_data_file_path = os.path.join(data_dir, 'test_data.csv')
test_data = pd.read_csv(test_data_file_path, parse_dates=['Timestamp']) # read test dataframe 
pseudo_test_data = test_data[test_data.answerCode==-1] # copy test_data to be filled with pseudo_labels  
pseudo_test_data = test_data.sort_values(by=['userID','Timestamp'], axis=0)

print('[  pseudo_test_data  ] : ', len(pseudo_test_data))

[  pseudo_labels  ] :  744


In [19]:
# update pseudo labeled data
for i, pseudo_label in enumerate(pseudo_labels):
    pseudo_test_data.answerCode[i] = pseudo_label
    
print('[  pseudo_test_data  ] : ', len(pseudo_test_data))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


[  pseudo_test_data  ] :  260114


In [6]:
train_data_file_path = os.path.join(data_dir, 'train_data.csv')
train_data = pd.read_csv(train_data_file_path, parse_dates=['Timestamp']) # read train dataframe 
train_data['pseudo'] = False
pseudo_test_data['pseudo'] = True
pseudo_labeled_data = pd.concat([train_data, pseudo_test_data])

print('[  train_data  ] : ', len(train_data))
print('[  pseudo_test_data  ] : ', len(pseudo_test_data))
print("-"*40)
print('[  pseudo_labeled_data  ] : ', len(pseudo_labeled_data))

[  train_data  ] :  2266586
[  pseudo_test_data  ] :  260114
----------------------------------------
[  pseudo_labeled_data  ] :  2526700


In [20]:
#save pseudo_labeled dataframe (pseudo_labeling.csv)
pseudo_labeled_data_file_path = os.path.join(data_dir, 'pseudo_labeling.csv')
pseudo_labeled_data.to_csv(pseudo_labeled_data_file_path, mode='w')